# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f44d88a2c10>
├── 'a' --> tensor([[-0.6344, -0.4627, -0.5508],
│                   [-1.7655, -0.9810,  1.0633]])
└── 'x' --> <FastTreeValue 0x7f44d88a28e0>
    └── 'c' --> tensor([[-1.0219,  1.9453, -0.8689, -0.7431],
                        [ 1.7156,  0.6123, -1.2573, -1.7966],
                        [ 2.2031,  1.9475,  0.8056,  1.0658]])

In [4]:
t.a

tensor([[-0.6344, -0.4627, -0.5508],
        [-1.7655, -0.9810,  1.0633]])

In [5]:
%timeit t.a

73.5 ns ± 0.127 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f44d88a2c10>
├── 'a' --> tensor([[-1.7000,  0.8757,  1.3652],
│                   [ 0.7086, -0.7701,  0.3615]])
└── 'x' --> <FastTreeValue 0x7f44d88a28e0>
    └── 'c' --> tensor([[-1.0219,  1.9453, -0.8689, -0.7431],
                        [ 1.7156,  0.6123, -1.2573, -1.7966],
                        [ 2.2031,  1.9475,  0.8056,  1.0658]])

In [7]:
%timeit t.a = new_value

73 ns ± 0.101 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6344, -0.4627, -0.5508],
               [-1.7655, -0.9810,  1.0633]]),
    x: Batch(
           c: tensor([[-1.0219,  1.9453, -0.8689, -0.7431],
                      [ 1.7156,  0.6123, -1.2573, -1.7966],
                      [ 2.2031,  1.9475,  0.8056,  1.0658]]),
       ),
)

In [10]:
b.a

tensor([[-0.6344, -0.4627, -0.5508],
        [-1.7655, -0.9810,  1.0633]])

In [11]:
%timeit b.a

66.5 ns ± 0.119 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-5.8073e-01,  2.3707e-04,  5.1319e-01],
               [-1.1686e+00,  2.5774e+00, -2.1661e-01]]),
    x: Batch(
           c: tensor([[-1.0219,  1.9453, -0.8689, -0.7431],
                      [ 1.7156,  0.6123, -1.2573, -1.7966],
                      [ 2.2031,  1.9475,  0.8056,  1.0658]]),
       ),
)

In [13]:
%timeit b.a = new_value

549 ns ± 5.66 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

886 ns ± 0.919 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 75.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

149 µs ± 948 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

150 µs ± 750 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f43f9959160>
├── 'a' --> tensor([[[-0.6344, -0.4627, -0.5508],
│                    [-1.7655, -0.9810,  1.0633]],
│           
│                   [[-0.6344, -0.4627, -0.5508],
│                    [-1.7655, -0.9810,  1.0633]],
│           
│                   [[-0.6344, -0.4627, -0.5508],
│                    [-1.7655, -0.9810,  1.0633]],
│           
│                   [[-0.6344, -0.4627, -0.5508],
│                    [-1.7655, -0.9810,  1.0633]],
│           
│                   [[-0.6344, -0.4627, -0.5508],
│                    [-1.7655, -0.9810,  1.0633]],
│           
│                   [[-0.6344, -0.4627, -0.5508],
│                    [-1.7655, -0.9810,  1.0633]],
│           
│                   [[-0.6344, -0.4627, -0.5508],
│                    [-1.7655, -0.9810,  1.0633]],
│           
│                   [[-0.6344, -0.4627, -0.5508],
│                    [-1.7655, -0.9810,  1.0633]]])
└── 'x' --> <FastTreeValue 0x7f43f9959f10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33 µs ± 86.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4401ed9490>
├── 'a' --> tensor([[-0.6344, -0.4627, -0.5508],
│                   [-1.7655, -0.9810,  1.0633],
│                   [-0.6344, -0.4627, -0.5508],
│                   [-1.7655, -0.9810,  1.0633],
│                   [-0.6344, -0.4627, -0.5508],
│                   [-1.7655, -0.9810,  1.0633],
│                   [-0.6344, -0.4627, -0.5508],
│                   [-1.7655, -0.9810,  1.0633],
│                   [-0.6344, -0.4627, -0.5508],
│                   [-1.7655, -0.9810,  1.0633],
│                   [-0.6344, -0.4627, -0.5508],
│                   [-1.7655, -0.9810,  1.0633],
│                   [-0.6344, -0.4627, -0.5508],
│                   [-1.7655, -0.9810,  1.0633],
│                   [-0.6344, -0.4627, -0.5508],
│                   [-1.7655, -0.9810,  1.0633]])
└── 'x' --> <FastTreeValue 0x7f4401d6f250>
    └── 'c' --> tensor([[-1.0219,  1.9453, -0.8689, -0.7431],
                        [ 1.7156,  0.6123, -1.2573, -1.7966],
                 

In [23]:
%timeit t_cat(trees)

31 µs ± 254 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.7 µs ± 66.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.0219,  1.9453, -0.8689, -0.7431],
                       [ 1.7156,  0.6123, -1.2573, -1.7966],
                       [ 2.2031,  1.9475,  0.8056,  1.0658]],
              
                      [[-1.0219,  1.9453, -0.8689, -0.7431],
                       [ 1.7156,  0.6123, -1.2573, -1.7966],
                       [ 2.2031,  1.9475,  0.8056,  1.0658]],
              
                      [[-1.0219,  1.9453, -0.8689, -0.7431],
                       [ 1.7156,  0.6123, -1.2573, -1.7966],
                       [ 2.2031,  1.9475,  0.8056,  1.0658]],
              
                      [[-1.0219,  1.9453, -0.8689, -0.7431],
                       [ 1.7156,  0.6123, -1.2573, -1.7966],
                       [ 2.2031,  1.9475,  0.8056,  1.0658]],
              
                      [[-1.0219,  1.9453, -0.8689, -0.7431],
                       [ 1.7156,  0.6123, -1.2573, -1.7966],
                       [ 2.2031,  1.9475,  0.8056,  1.0658]],

In [26]:
%timeit Batch.stack(batches)

84.3 µs ± 100 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.0219,  1.9453, -0.8689, -0.7431],
                      [ 1.7156,  0.6123, -1.2573, -1.7966],
                      [ 2.2031,  1.9475,  0.8056,  1.0658],
                      [-1.0219,  1.9453, -0.8689, -0.7431],
                      [ 1.7156,  0.6123, -1.2573, -1.7966],
                      [ 2.2031,  1.9475,  0.8056,  1.0658],
                      [-1.0219,  1.9453, -0.8689, -0.7431],
                      [ 1.7156,  0.6123, -1.2573, -1.7966],
                      [ 2.2031,  1.9475,  0.8056,  1.0658],
                      [-1.0219,  1.9453, -0.8689, -0.7431],
                      [ 1.7156,  0.6123, -1.2573, -1.7966],
                      [ 2.2031,  1.9475,  0.8056,  1.0658],
                      [-1.0219,  1.9453, -0.8689, -0.7431],
                      [ 1.7156,  0.6123, -1.2573, -1.7966],
                      [ 2.2031,  1.9475,  0.8056,  1.0658],
                      [-1.0219,  1.9453, -0.8689, -0.7431],
                   

In [28]:
%timeit Batch.cat(batches)

153 µs ± 226 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

386 µs ± 5.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
